In [2]:
from functools import reduce
from pathlib import Path  # NOTE For testing
import numpy
import pickle  # NOTE For testing

# NOTE For testing
listArrayCurveLocationsAnalyzed: list[numpy.ndarray[tuple[int, int], numpy.dtype[numpy.uint64]]] = pickle.loads(Path("listArrayCurveLocationsAnalyzed.pkl").read_bytes())
"""NOTE data overview
index=0	(194, 2)	194=Unique curveLocations
index=1	(276, 2)	276=Unique curveLocations
index=2	(276, 2)	276=Unique curveLocations
index=3	(207, 2)	132=Unique curveLocations

953 Total rows
470 Total unique curveLocations

Created by running `A000682(15)` and pickling `listArrayCurveLocationsAnalyzed` at the end of the while loop when `bridges==7`.
"""

indexDistinctCrossings: int = 0
indexCurveLocations: int = 1

"""NOTE What, why, and how:

Create a function that receives one parameter, `listArrayCurveLocationsAnalyzed`, and returns one value, `arrayAggregated`.

Collect "coordinates" to create a low-quality look-up system. Given a value for `curveLocations`, for each
`arrayCurveLocationsAnalyzed`, I get all indices of that `curveLocations`. Then I sum the list of `distinctCrossings` at those
indices.

I have a few goals with this approach.
- I believe my clumsy prototype can be improved to be faster and less memory expensive.
- I _think_ the memory requirements of a coordinate system are less expensive than moving the actual data.
- I _think_ the process of collecting coordinates is faster than moving data.
- I want to create `arrayAggregated` from `listArrayCurveLocationsAnalyzed` by moving/computing the data with EXACTLY ONE
vectorized operation. I _feel_ that is possible if I can structure the coordinate information in a clever way.
"""

# NOTE Purpose and effect: Create a list of coordinates as a tuple
listOfCoordinates: list[tuple[numpy.ndarray[tuple[int, ...], numpy.dtype[numpy.intp]], numpy.ndarray[tuple[int, ...], numpy.dtype[numpy.uint64]], int]] = []
for index, arrayCurveLocationsAnalyzed in enumerate(listArrayCurveLocationsAnalyzed):
	# NOTE Development: I am using `index` at the moment. I hope that vectorizing will obsolete it.
	arrayCurveLocationsUniqueHere , indicesDistinctCurvesToSum = numpy.unique(arrayCurveLocationsAnalyzed[:, indexCurveLocations], return_inverse=True)
	# NOTE Development: the tuple must be indexed, so to make it easier on myself, mirror the array indexing.
	listOfCoordinates.append((indicesDistinctCurvesToSum, arrayCurveLocationsUniqueHere, index))

# NOTE Purpose and effect: This is the correct shape for `arrayAggregated`, and each column has a unique `curveLocations` value.
arrayAggregated: numpy.ndarray[tuple[int, ...], numpy.dtype[numpy.uint64]] = numpy.tile(
	A=reduce(numpy.union1d, [tupleCoordinates[indexCurveLocations] for tupleCoordinates in listOfCoordinates])
	, reps=(2, 1)
).T

# NOTE Purpose and effect: Literally zero-out the values for `distinctCrossings`.
arrayAggregated[:, indexDistinctCrossings] = 0
# Above this line is surprisingly fast --------------------------------------------------------------------------



In [ ]:
# I _feel_ like I don't need to loop based on `arrayAggregated[:, indexCurveLocations]`. I feel like I can make selectors using
# each `arrayCurveLocationsAnalyzed`. Then select the selectors so they align with `arrayAggregated[:, indexCurveLocations]`. If
# that were to happen, the summation would happen in one, vectorized action.

indexForLists = 3
listSelectors = [0] * len(arrayAggregated[:, indexCurveLocations])
# Put the coordinates of the selectors in the same order as `arrayAggregated[:, indexCurveLocations]`
for indexACLA, curveLocations in enumerate(listOfCoordinates[indexForLists][indexCurveLocations]):
    indexAADC = numpy.searchsorted(arrayAggregated[:, indexCurveLocations], curveLocations)
    listSelectors[indexAADC] = listOfCoordinates[indexForLists][indexDistinctCrossings] == indexACLA
    listSelectors[indexAADC] = (indexForLists, indexACLA)

# This `listOfCoordinates[indexForLists][indexDistinctCrossings] == sIndex` is a full sized boolean array, which I don't need yet.
# This `(listOfCoordinates[indexForLists][indexDistinctCrossings], sIndex)` has the data necessary to create the boolean array at the moment I need it.

In [ ]:
# Hide this code from Copilot because it constantly tries to delete everything.

	# for indexArrayAggregatedRow in range(len(arrayAggregated[:, indexCurveLocations])):
	# 	curveLocations: numpy.uint64 = arrayAggregated[indexArrayAggregatedRow, indexCurveLocations]

	# 	for index in range(len(listOfCoordinates)):
	# 		insertIndex: numpy.intp = numpy.searchsorted(listOfCoordinates[index][indexCurveLocations], curveLocations)
	# 		if insertIndex < listOfCoordinates[index][indexCurveLocations].size and listOfCoordinates[index][indexCurveLocations][insertIndex] == curveLocations:
	# 			selector: numpy.ndarray[tuple[int], numpy.dtype[numpy.bool_]] = listOfCoordinates[index][indexDistinctCrossings] == insertIndex
	# 			arrayAggregated[indexArrayAggregatedRow, indexDistinctCrossings] += listArrayCurveLocationsAnalyzed[index][selector, indexDistinctCrossings].sum(dtype=numpy.uint64)

In [ ]:

# NOTE Purpose and effect: for each `curveLocations` in `arrayAggregated`, sum the `distinctCrossings` across all `arrayCurveLocationsAnalyzed`.
for indexArrayAggregatedRow in range(len(arrayAggregated[:, indexCurveLocations])):
	curveLocations = arrayAggregated[indexArrayAggregatedRow, indexCurveLocations]

	for index in range(len(listOfCoordinates)):
		if curveLocations in listOfCoordinates[index][indexCurveLocations]:
			selector: numpy.ndarray[tuple[int], numpy.dtype[numpy.bool_]] = listOfCoordinates[index][indexDistinctCrossings] == numpy.searchsorted(listOfCoordinates[index][indexCurveLocations], curveLocations)
			# NOTE Development: A little voice in my head is telling me `numpy.add.reduce`.
			arrayAggregated[indexArrayAggregatedRow, indexDistinctCrossings] += numpy.sum(numpy.atleast_1d(listArrayCurveLocationsAnalyzed[index][selector, indexDistinctCrossings]), initial=0)

# NOTE A very simple test
print(arrayAggregated[2] == numpy.array([156, 60], numpy.uint64))

# Proof of concept, sum the same `curveLocations` across all arrays

In [ ]:
# # NOTE Proof of concept, sum the same `curveLocations` across all arrays START-----------------------------------------------------------------------
# print(TESTindexOfOneRow := 2)
# print(arrayAggregated[0:10, indexCurveLocations])
# print(arrayAggregated[TESTindexOfOneRow, indexCurveLocations])
# TESTcurveLocationsUnique = arrayAggregated[TESTindexOfOneRow, indexCurveLocations]

# for index in range(len(listOfCoordinates)):
# 	print(f"{index = }")
# 	print(listOfCoordinates[index][indexCurveLocations][0:10])
# 	if TESTcurveLocationsUnique in listOfCoordinates[index][indexCurveLocations]:
# 		TESTindexArrayCurveLocationsUniqueHere: int = listOfCoordinates[index][indexCurveLocations].tolist().index(TESTcurveLocationsUnique)
# 		print(f"{TESTindexArrayCurveLocationsUniqueHere=}")
# 		print(listOfCoordinates[index][indexDistinctCrossings][0:10])
# 		selectTESTindexArrayCurveLocationsUniqueHere: numpy.ndarray[tuple[int], numpy.dtype[numpy.bool_]] = listOfCoordinates[index][indexDistinctCrossings] == TESTindexArrayCurveLocationsUniqueHere
# 		print(selectTESTindexArrayCurveLocationsUniqueHere[0:10])
# 		print(listArrayCurveLocationsAnalyzed[index][0:10, indexDistinctCrossings])
# 		TESTarrayDistinctCrossingsOfTESTcurveLocationsUnique: numpy.ndarray[tuple[int, ...], numpy.dtype[numpy.uint64]] = numpy.atleast_1d(listArrayCurveLocationsAnalyzed[index][selectTESTindexArrayCurveLocationsUniqueHere, indexDistinctCrossings])
# 		print(f"{TESTarrayDistinctCrossingsOfTESTcurveLocationsUnique=}")
# 		# NOTE Development: A little voice in my head is telling me `numpy.add.reduce`.
# 		TESTsumDistinctCrossings: numpy.uint64 = numpy.sum(TESTarrayDistinctCrossingsOfTESTcurveLocationsUnique)
# 		print(f"{TESTsumDistinctCrossings=}")
# 		arrayAggregated[TESTindexOfOneRow, indexDistinctCrossings] += TESTsumDistinctCrossings

# print(arrayAggregated[TESTindexOfOneRow])
# # NOTE Proof of concept, sum the same `curveLocations` across all arrays END -----------------------------------------------------------------------

## Analysis of proof of concept loop for one row

```python
2
[ 22  41  60  63 326 338 364 367 376 379]
60 # `curveLocations` value for which we want the sum of `distinctCrossings` across all `arrayCurveLocationsAnalyzed`.

index = 0 # i.e., `listArrayCurveLocationsAnalyzed[0]`
[  63  367  379  671  695  975  987  999 1011 1023]
# `curveLocations` 60 is not in the list above, so we must skip it.

index = 1
[ 41  60  63 364 367 376 379 382 649 668]
TESTindexArrayCurveLocationsUniqueHere=1 # `curveLocations` 60 is at index 1 in the list above.
[ 0  1  2  8 11  9 10 12 13 14] # A list of indices
[False  True False False False False False False False False] # Selector is True where list-element matches `TESTindexArrayCurveLocationsUniqueHere`
[ 66  64 108  64 108  26  14  14  50 154] # Selector is applied to this list
TESTarrayDistinctCrossingsOfTESTcurveLocationsUnique=array([64], dtype=uint64)
TESTsumDistinctCrossings=np.uint64(64)

index = 2
[ 22  60  63 326 338 364 367 376 379 382]
TESTindexArrayCurveLocationsUniqueHere=1
[0 3 4 1 2 5 7 6 8 9]
[False False False  True False False False False False False]
[ 66  64 108  64 108  26  14  14  50 154]
TESTarrayDistinctCrossingsOfTESTcurveLocationsUnique=array([64], dtype=uint64)
TESTsumDistinctCrossings=np.uint64(64)

index = 3
[ 22  41  60  63 326 338 364 367 376 379]
TESTindexArrayCurveLocationsUniqueHere=2
[0 1 2 2 3 4 5 4 6 6]
[False False  True  True False False False False False False]
[108 108  14  14  50  14  50  78   2   2]
TESTarrayDistinctCrossingsOfTESTcurveLocationsUnique=array([14, 14], dtype=uint64)
TESTsumDistinctCrossings=np.uint64(28)
[156  60]

156 = 64 + 64 + 28
```

# `_aggregate` function

In [ ]:
from functools import reduce
import numpy

# NOTE `arrayCurveLocationsAnalyzed` and `arrayAggregated`: Always use semantic index identifiers: Never hardcode the indices.
indexDistinctCrossings: int = 0
indexCurveLocations: int = 1

def _aggregate(listArrayCurveLocationsAnalyzed: list[numpy.ndarray[tuple[int, int], numpy.dtype[numpy.uint64]]]) -> numpy.ndarray[tuple[int, int], numpy.dtype[numpy.uint64]]:
	"""Common aggregation tasks."""
	# NOTE Purpose and effect: Create a list of coordinates as a tuple
	listOfCoordinates: list[tuple[numpy.ndarray[tuple[int, ...], numpy.dtype[numpy.intp]], numpy.ndarray[tuple[int, ...], numpy.dtype[numpy.uint64]], int]] = []
	for index, arrayCurveLocationsAnalyzed in enumerate(listArrayCurveLocationsAnalyzed):
		# NOTE Development: I am using `index` at the moment. I hope that vectorizing will obsolete it.
		arrayCurveLocationsUniqueHere , indicesDistinctCurvesToSum = numpy.unique(arrayCurveLocationsAnalyzed[:, indexCurveLocations], return_inverse=True)
		# NOTE Development: the tuple must be indexed, so to make it easier on myself, mirror the array indexing.
		listOfCoordinates.append((indicesDistinctCurvesToSum, arrayCurveLocationsUniqueHere, index))

	# NOTE Purpose and effect: This is the correct shape for `arrayAggregated`, and each column has a unique `curveLocations` value.
	arrayAggregated: numpy.ndarray[tuple[int, ...], numpy.dtype[numpy.uint64]] = numpy.tile(
		A=reduce(numpy.union1d, [tupleCoordinates[indexCurveLocations] for tupleCoordinates in listOfCoordinates])
		, reps=(2, 1)
	).T

	# NOTE Purpose and effect: Literally zero-out the values for `distinctCrossings`.
	arrayAggregated[:, indexDistinctCrossings] = 0

	# I _feel_ like I don't need to loop based on `arrayAggregated[:, indexCurveLocations]`. I feel like I can make selectors using
	# each `arrayCurveLocationsAnalyzed`. Then select the selectors so they align with `arrayAggregated[:, indexCurveLocations]`. If
	# that were to happen, the summation would happen in one, vectorized action.

	# NOTE Purpose and effect: for each `curveLocations` in `arrayAggregated`, sum the `distinctCrossings` across all `arrayCurveLocationsAnalyzed`.
	for indexArrayAggregatedRow in range(len(arrayAggregated[:, indexCurveLocations])):
		curveLocations = arrayAggregated[indexArrayAggregatedRow, indexCurveLocations]

		for index in range(len(listOfCoordinates)):
			if curveLocations in listOfCoordinates[index][indexCurveLocations]:
				selector: numpy.ndarray[tuple[int], numpy.dtype[numpy.bool_]] = listOfCoordinates[index][indexDistinctCrossings] == numpy.searchsorted(listOfCoordinates[index][indexCurveLocations], curveLocations)
				# NOTE Development: A little voice in my head is telling me `numpy.add.reduce`.
				arrayAggregated[indexArrayAggregatedRow, indexDistinctCrossings] += numpy.sum(numpy.atleast_1d(listArrayCurveLocationsAnalyzed[index][selector, indexDistinctCrossings]), initial=0)

	return arrayAggregated